In [1]:
# import packages

import numpy as np
from numpy import random

# import r packages (and functions) if needed

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
#rpackages.importr("clusterGeneration")
rpackages.importr("mpower")
#cluster_generation = robjects.packages.importr("clusterGeneration")
mpower = robjects.packages.importr("mpower")

In [2]:
# Fully Synthetic Data Sets (Set up from Künzel et al.)

# 1: Simulate the d-dimensional X.
# 2: Create Potential Outcomes Y(1) and Y(0).
# 3: Simulate Treatment Assignments trough W.

In [3]:
# 1: Simulate the d-dimensional X. #


# Setup parameters
d = 5 # TODO: set dimension according to setup
N = 10 # TODO: set sample size according to setup

# X Correlation matrix and mean
mean = np.zeros(d) # TODO: set mean according to setup
cov = np.array(mpower.cvine(d=d, alpha = 0.5, beta = 0.5)) # TODO: set cov according to setup

# Simulate X
X = random.multivariate_normal(mean=mean, cov=cov, size=N, check_valid='warn')

In [4]:
# Check X
X

array([[ 0.30406862,  1.7801453 ,  0.6997692 ,  1.05736195, -1.42485726],
       [ 0.45293072, -2.23144037, -1.01904289,  1.08417423, -0.56909043],
       [-0.61961636,  0.81674151,  0.61843623, -0.82075735,  0.98068663],
       [-1.16624963,  0.46885978,  0.92428784, -0.14893294,  0.61415847],
       [-0.43907415, -0.03808029,  0.50829344,  0.07482647, -1.29855536],
       [-1.95979695,  1.99700116,  2.03644761, -2.91456776,  0.52377819],
       [-0.2960675 ,  0.31951868,  0.06947292, -1.56045671,  1.30700133],
       [-0.47933482,  0.49233353,  0.33106926, -0.77413148,  1.14266221],
       [ 0.9282673 ,  1.28447366, -0.13461971,  0.68668656, -0.38495443],
       [ 0.72909416,  0.3158631 , -0.10476353,  1.25213071, -1.47461883]])

In [5]:
# Check cov
cov

array([[ 1.        , -0.39501135, -0.90439808,  0.43122745,  0.1128078 ],
       [-0.39501135,  1.        ,  0.70401778, -0.34423844, -0.23647799],
       [-0.90439808,  0.70401778,  1.        , -0.32834201, -0.36935233],
       [ 0.43122745, -0.34423844, -0.32834201,  1.        , -0.49208684],
       [ 0.1128078 , -0.23647799, -0.36935233, -0.49208684,  1.        ]])

In [6]:
# 2: Create Potential Outcomes Y(1) and Y(0).

# Simulate errors, FIX
e_0 = random.normal(loc=0.0,scale=1.0,size=N)
e_1 = random.normal(loc=0.0,scale=1.0,size=N)

# Just to make it work, create betas
beta_0 = [2,3,-3,1,4] # TODO: delete or change
beta_1 = [1,1,0,-5,3] # TODO: delete or change

# Create Response Functions
mu_0 = np.matmul(X,beta_0) # TODO: change according to setup
mu_1 = np.matmul(X,beta_1) # TODO: change according to setup

# Creat Potential Outcomes, FIX
Y_0 = mu_0 + e_0
Y_1 = mu_1 + e_1

In [7]:
# quick check Y_0
Y_0

array([ 1.04303442, -3.79811164,  1.72353384, -1.72133214, -8.28021499,
       -5.24568638,  3.15895282,  4.69914253,  7.95188519, -1.37315289])

In [8]:
# Quick Check Y_1
Y_1

array([-8.41530723, -8.7163901 ,  6.8355947 ,  2.84825891, -3.85210517,
       17.14287102, 12.72436134,  6.34962274, -1.46360217, -9.59764604])

In [9]:
# 3: Simulate Treatment Assignments trough W.

# Set propensity score
e_x = 0.5 # TODO: change from setup

# Simulate Treatment Assignment, FIX
W = random.binomial(size=N, n=1, p=e_x)

# Create Observed Outcome, FIX
ones = np.ones(N)
Y = np.multiply(W,Y_1) + np.multiply(ones-W,Y_0)

In [10]:
# check out observed outcomes
Y

array([ 1.04303442, -3.79811164,  1.72353384, -1.72133214, -3.85210517,
       -5.24568638, 12.72436134,  4.69914253,  7.95188519, -1.37315289])